### (Execute if your running in Binder)

In [ ]:
# Uncomment these lines

# !pip install pandas
# !pip install openpyxl
# !pip install networkx

# HS Conversion Tables

In [8]:
import pandas as pd
import os
from pathlib import Path
import platform
import networkx as nx

C:\Users\Usuario\Desktop\CEP\HSConversionTables


## Get Subgraph

In [11]:
HSTABLES = "https://unstats.un.org/unsd/trade/classifications/tables/CompleteCorrelationsOfHS-SITC-BEC_20170606.xlsx"
HS = ['HS92', 'HS96', 'HS02', 'HS07', 'HS12', 'HS17'] #should be updated if changed

import functools
from datetime import datetime

def return_used_versions(start_year = None, end_year=None):
    if start_year==None: 
        start_year = 1992
    if end_year==None: 
        end_year = datetime.now().year
    if start_year < 1992 or end_year<start_year or end_year>datetime.now().year: 
        print("Uncorrect years")
    else: 
        years = [1992, 1996, 2002, 2007, 2012, 2017]
        versions = ['HS92', 'HS96', 'HS02', 'HS07', 'HS12', 'HS17']
        version_dict = {k:v for k,v in zip(years, versions)}
        start_v = functools.reduce(lambda a,b: a if a<=start_year else b, list(reversed(years)))
        end_v = functools.reduce(lambda a,b: a if a<=end_year else b, list(reversed(years)))
        years_list = years[years.index(start_v):years.index(end_v)+1]
        return [version_dict[x] for x in years_list]

def find_homogeneous_serie(position=None, start_year=None, end_year=None):
    if position: 
        cols = return_used_versions(start_year, end_year)
        print(f"Loading HS Correlations Tables\n")
        data = pd.read_excel(HSTABLES,
                            usecols= HS,
                            dtype = {v:'object' for v in HS}).dropna()
        data = data[cols].drop_duplicates()
        versions = [v for v in cols if data[v].str.contains(position).any()]
        for col in cols:
            data[col] = data[col].apply(lambda x: col+"-"+str(int(x)).zfill(6))
        connections = []
        for i in range(len(cols)-1): 
            temp_tup_list = list(data[[cols[i], cols[i+1]]].drop_duplicates().itertuples(index=False, name=None))
            connections = connections + temp_tup_list 
        G = nx.Graph()
        G.add_edges_from(connections)
        positions = sorted(nx.node_connected_component(G, versions[-1]+"-"+position))
        print(positions)
        #return positions

In [12]:
find_homogeneous_serie("010111")

Loading HS Correlations Tables

['HS02-010110', 'HS02-010190', 'HS07-010110', 'HS07-010190', 'HS12-010121', 'HS12-010129', 'HS12-010130', 'HS12-010190', 'HS17-010121', 'HS17-010129', 'HS17-010130', 'HS17-010190', 'HS92-010111', 'HS92-010119', 'HS92-010120', 'HS96-010111', 'HS96-010119', 'HS96-010120']


## Trade-off

In [13]:
import functools
from datetime import datetime
import numpy as np

HSTABLES = "https://unstats.un.org/unsd/trade/classifications/tables/CompleteCorrelationsOfHS-SITC-BEC_20170606.xlsx"

HS = ['HS92', 'HS96', 'HS02', 'HS07', 'HS12', 'HS17'] #should be updated if changed

def return_used_versions(start_year = None, end_year=None):
    years = [1992, 1996, 2002, 2007, 2012, 2017]
    versions = HS
    version_dict = {k:v for k,v in zip(years, versions)}
    start_v = functools.reduce(lambda a,b: a if a<=start_year else b, list(reversed(years)))
    end_v = functools.reduce(lambda a,b: a if a<=end_year else b, list(reversed(years)))
    years_list = years[years.index(start_v):years.index(end_v)+1]
    return [version_dict[x] for x in years_list]

def recursive_trade_off(df, position, k): 
    
    if k>=0:
        currcols = df.columns.tolist()[k:]
        print(f"Evaluating graph object with {','.join(currcols)} versions\n")
        data = df[currcols].drop_duplicates()
        connections = []
        for i in range(len(currcols)-1): 
            temp_tup_list = list(data[[currcols[i], currcols[i+1]]].drop_duplicates().itertuples(index=False, name=None))
            connections = connections + temp_tup_list
        G = nx.Graph()
        G.add_edges_from(connections)
        try:
            positions = sorted(nx.node_connected_component(G, currcols[-1]+"-"+position))
            if len(positions)==len(currcols): 
                return recursive_trade_off(df, position, k-1)
            else: 
                print("Evaluation finished\n")
                print(f"The {','.join(currcols[1:])} versions support your position with no precision loss")
        except:
            print(f"Position {position} not founded")
    else:
        print(f"The {','.join(df.columns.tolist())} versions support your position with no precision loss")
        
    
def trade_off(position=None, start_year=None, end_year=None):
    if start_year==None: 
        start_year = 1992
    if end_year==None: 
        end_year = datetime.now().year
    if start_year < 1992 or end_year<start_year or end_year>datetime.now().year: 
        print("Uncorrect years")
    
    if position: 
        versions = return_used_versions(start_year, end_year)
        print(f"Period between {start_year} and {end_year} contains the {','.join(versions)} versions\n") 
        print(f"Loading HS Correlations Tables\n")
        
        data = pd.read_excel(HSTABLES,
                            usecols= HS,
                            dtype = {v:'object' for v in HS}).dropna()
        versions = [v for v in versions if data[v].str.contains(position).any()]
        
        if len(versions)>0: 
            print(f"The position {position} was included in the {','.join(versions)} versions\n")
            data = data[versions].drop_duplicates()
            for col in data.columns:
                data[col] = data[col].apply(lambda x: col+"-"+str(int(x)).zfill(6))
            k=len(versions)-2
            print(f"Evaluating maximum period with no precision loss for position {position}\n")
            return recursive_trade_off(data, position, k)
        else:
            print(f"Position {position} not founded in that period")
    else:
        print("Please define a position")

In [14]:
trade_off("010130")

Period between 1992 and 2022 contains the HS92,HS96,HS02,HS07,HS12,HS17 versions

Loading HS Correlations Tables

The position 010130 was included in the HS12,HS17 versions

Evaluating maximum period with no precision loss for position 010130

Evaluating graph object with HS12,HS17 versions

The HS12,HS17 versions support your position with no precision loss
